<a href="https://colab.research.google.com/github/Breakpoint23/imc_2022_RRL/blob/Meet/Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
mkdir ~/.kaggle

/content


In [ ]:
cp kaggle.json ~/.kaggle

In [ ]:
!kaggle competitions download -c image-matching-challenge-2022

100% 2.30G/2.31G [01:11<00:00, 53.3MB/s]
100% 2.31G/2.31G [01:11<00:00, 34.7MB/s]


In [ ]:
!unzip image-matching-challenge-2022.zip

In [ ]:
import pandas as pd
import numpy as np

In [72]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchsummary import summary
import numpy as np
class SuperPointNet(torch.nn.Module):
  """ Pytorch definition of SuperPoint Network. """
  def __init__(self):
    super(SuperPointNet, self).__init__()
    self.relu = torch.nn.ReLU(inplace=True)
    self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    c1, c2, c3, c4, c5, d1 = 64, 64, 128, 128, 256, 256
    # Shared Encoder.
    self.conv1a = torch.nn.Conv2d(1, c1, kernel_size=3, stride=1, padding=1)
    self.conv1b = torch.nn.Conv2d(c1, c1, kernel_size=3, stride=1, padding=1)
    self.conv2a = torch.nn.Conv2d(c1, c2, kernel_size=3, stride=1, padding=1)
    self.conv2b = torch.nn.Conv2d(c2, c2, kernel_size=3, stride=1, padding=1)
    self.conv3a = torch.nn.Conv2d(c2, c3, kernel_size=3, stride=1, padding=1)
    self.conv3b = torch.nn.Conv2d(c3, c3, kernel_size=3, stride=1, padding=1)
    self.conv4a = torch.nn.Conv2d(c3, c4, kernel_size=3, stride=1, padding=1)
    self.conv4b = torch.nn.Conv2d(c4, c4, kernel_size=3, stride=1, padding=1)
    # Detector Head.
    self.convPa = torch.nn.Conv2d(c4, c5, kernel_size=3, stride=1, padding=1)
    self.convPb = torch.nn.Conv2d(c5, 65, kernel_size=1, stride=1, padding=0)
    # Descriptor Head.
    self.convDa = torch.nn.Conv2d(c4, c5, kernel_size=3, stride=1, padding=1)
    self.convDb = torch.nn.Conv2d(c5, d1, kernel_size=1, stride=1, padding=0)

  def forward(self, x):
    """ Forward pass that jointly computes unprocessed point and descriptor
    tensors.
    Input
      x: Image pytorch tensor shaped N x 1 x H x W.
    Output
      semi: Output point pytorch tensor shaped N x 65 x H/8 x W/8.
      desc: Output descriptor pytorch tensor shaped N x 256 x H/8 x W/8.
    """
    # Shared Encoder.
    x = self.relu(self.conv1a(x))
    x = self.relu(self.conv1b(x))
    x = self.pool(x)
    x = self.relu(self.conv2a(x))
    x = self.relu(self.conv2b(x))
    x = self.pool(x)
    x = self.relu(self.conv3a(x))
    x = self.relu(self.conv3b(x))
    x = self.pool(x)
    x = self.relu(self.conv4a(x))
    x = self.relu(self.conv4b(x))
    # Detector Head.
    cPa = self.relu(self.convPa(x))
    semi = self.convPb(cPa)
    # Descriptor Head.
    cDa = self.relu(self.convDa(x))
    desc = self.convDb(cDa)
    dn = torch.norm(desc, p=2, dim=1) # Compute the norm.
    desc = desc.div(torch.unsqueeze(dn, 1)) # Divide by norm to normalize.
    return semi, desc

class FunRec(nn.Module):
  """
  How to reconstruct fundamental matrix
  Need 12 parameters before the reconstruction
  each image's output is a tensor of shape (N, 65+256, H/8, W/8)
  for two images ,shape would be (N,2*(65+256), H/8, W/8)
  (f1,cx1,cy1,f2,cx2,cy2,rx,ry,rz,tx,ty,tz)
  F=K2^-T*t*R*K1^-1
  """
  def __init__(self):
    super(FunRec, self).__init__()
    self.device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.relu=nn.ReLU(inplace=True)
    self.net = SuperPointNet()
    self.net.load_state_dict(torch.load('superpoint_v1.pth'))
    self.net.to(self.device)
    self.input_norm=nn.BatchNorm2d(num_features=1)
    self.conv1a=nn.Conv2d(in_channels=2*(65+256),out_channels=64,kernel_size=3,stride=1)
    #outshape=(N,64,H/8,W/8)
    self.batchnorm1=nn.BatchNorm2d(num_features=64)
    self.conv2a=nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3,stride=1)
    #outshape=(N,32,H/8,W/8)
    self.batchnorm2=nn.BatchNorm2d(num_features=32)
    self.dense1=nn.Linear(in_features=int(32*24*28),out_features=int(8*24*28))
    self.dense2=nn.Linear(in_features=int(8*24*28),out_features=12)

  def forward(self,x):
    """
    x1: (N,65+256,H/8,W/8)
    x2: (N,65+256,H/8,W/8)
    """
    
    x1=torch.unsqueeze(x[:,0,:,:],1)
    x2=torch.unsqueeze(x[:,1,:,:],1)
    
    x1=self.input_norm(x1)
    x2=self.input_norm(x2)
    x1semi,x1desc=self.net(x1)
    x2semi,x2desc=self.net(x2)
    #print(f'x1semi.shape={x1semi.shape},x1desc.shape={x1desc.shape},x2semi.shape={x2semi.shape},x2desc.shape={x2desc.shape}')
    x1=torch.concat((x1semi,x2semi),1)
    x2=torch.concat((x1desc,x2desc),1)
    x=torch.cat((x1,x2),1)
    x=self.conv1a(x)
    x=self.relu(x)
    x=self.batchnorm1(x)
    x=self.conv2a(x)
    x=self.relu(x)
    x=self.batchnorm2(x)
    #print(x.shape,32*(224/8)*(256/8),32*24*28)
    x=x.view(x.shape[0],-1)
    
    x=self.dense1(x)
    x=self.relu(x)
    x=self.dense2(x)
    tensor_0=torch.zeros(1,requires_grad=True).to(self.device)
    tensor_1=torch.ones(1,requires_grad=True).to(self.device)
    
    for i,sample in enumerate(x):
      #print(sample)
      f1,cx1,cy1,f2,cx2,cy2,rx,ry,rz,tx,ty,tz=sample
      f1,cx1,cy1,f2,cx2,cy2,rx,ry,rz,tx,ty,tz=f1.reshape(1),cx1.reshape(1),cy1.reshape(1),f2.reshape(1),cx2.reshape(1),cy2.reshape(1),rx.reshape(1),ry.reshape(1),rz.reshape(1),tx.reshape(1),ty.reshape(1),tz.reshape(1)
      #print(torch.stack([f1,tensor_0,cx1]))
      K1=torch.stack([torch.stack([f1,tensor_0,cx1]),torch.stack([tensor_0,f1,cy1]),torch.stack([tensor_0,tensor_0,tensor_1])]).reshape(3,3)
      K2=torch.stack([torch.stack([f2,tensor_0,cx2]),torch.stack([tensor_0,f2,cy2]),torch.stack([tensor_0,tensor_0,tensor_1])]).reshape(3,3)
      Rx=torch.stack([torch.stack([tensor_1,tensor_0,tensor_0]),torch.stack([tensor_0,torch.cos(rx),-torch.sin(rx)]),torch.stack([tensor_0,torch.sin(rx),torch.cos(rx)])]).reshape(3,3)
      Ry=torch.stack([torch.stack([torch.cos(ry),tensor_0,torch.sin(ry)]),torch.stack([tensor_0,tensor_1,tensor_0]),torch.stack([-torch.sin(ry),tensor_0,torch.cos(ry)])]).reshape(3,3)
      Rz=torch.stack([torch.stack([torch.cos(rz),-torch.sin(rz),tensor_0]),torch.stack([torch.sin(rz),torch.cos(rz),tensor_0]),torch.stack([tensor_0,tensor_0,tensor_1])]).reshape(3,3)
      tx=torch.stack([torch.stack([tensor_0,-tz,ty]),torch.stack([tz,tensor_0,-tx]),torch.stack([-ty,tx,tensor_0])]).reshape(3,3)
      R=torch.matmul(torch.matmul(Rx,Ry),Rz)
      sample_f=torch.matmul(torch.matmul(torch.inverse(K2),tx),torch.matmul(R,torch.inverse(K1)))
      sample_f=sample_f.reshape(1,9)
      if i==0:
        y=sample_f
      else:
        y=torch.cat((y,sample_f),0)
      #print(y.shape)
    y=y.reshape(y.shape[0],9)
    y=y/torch.max(y)
    return y


In [60]:
import cv2 as cv
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import os,time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
# path to training data

path='train'

#list of directories


X1=np.empty((1,1,224,256))
X2=np.empty((1,1,224,256))
y=np.empty((1,9))

places=os.listdir(path)
for place in places:
    if '.' in place:
        places.remove(place)
prev_time=time.time()    
for place in places:
    pairs=pd.read_csv(path+f'/{place}/pair_covisibility.csv')
    pairs=pairs[pairs['covisibility']>0.1]
    pairs=pairs.reset_index()
    x1=[]
    x2=[]
    Y=[]
        
    for index,pair in pairs.iterrows():
        if len(x1)>2000:
            break
        try:                
            ids=pair['pair'].split('-')
            img1=cv.imread(path+f'/{place}/images/{ids[0]}.jpg',cv.IMREAD_GRAYSCALE)
                #print(img1)
            if img1.shape[0]>img1.shape[1]:
                img1=cv.rotate(img1,cv.ROTATE_90_CLOCKWISE)
            img2=cv.imread(path+f'/{place}/images/{ids[1]}.jpg',cv.IMREAD_GRAYSCALE)
            if img2.shape[0]>img2.shape[1]:
                img2=cv.rotate(img2,cv.ROTATE_90_CLOCKWISE)
            img1,img2=cv.resize(img1,(256,224)),cv.resize(img2,(256,224))
            img1,img2=np.array(img1),np.array(img2)
            img1,img2=img1.reshape(1,224,256),img2.reshape(1,224,256)
            x1.append(img1)
            x2.append(img2)
                #X1,X2=np.concatenate((X1,img1),axis=0),np.concatenate((X2,img2),axis=0)
            Fun_mat=pair['fundamental_matrix']
            Fun_mat=Fun_mat.split(' ')
            Fun_mat=np.array(Fun_mat).astype(float)
            Fun_mat=Fun_mat.reshape(1,9)
            Y.append(Fun_mat)
            if time.time()-prev_time>10:
                print(f'{place} {index}/{len(pairs)}')
                prev_time=time.time()
                #y=np.concatenate((y,Fun_mat),axis=0)
        except Exception as e:
              print(e)
              break
    x1=np.array(x1)
    x2=np.array(x2)
    Y=np.array(Y).reshape(-1,9)   
    X1=np.concatenate((X1,np.array(x1)),axis=0)
    X2=np.concatenate((X2,np.array(x1)),axis=0)
    y=np.concatenate((y,Y),axis=0)
    print(place,x1.shape,x2.shape,Y.shape)
    break

KeyboardInterrupt: ignored

In [ ]:


from sklearn import model_selection
import torch
import torch.nn as nn
import numpy as np
import cv2 as cv
import os

try:
  os.mkdir(model_selection)
except Exception as e:
  print(e)

from sklearn.metrics import confusion_matrix,classification_report

import pickle
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

places=os.listdir(path)
for place in places:
    if '.' in place:
      places.remove(place)

def train():
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    losses=[]
    #X1=np.array(X1)
    #X2=np.array(X2)
    #y=np.array(y).reshape(-1,9)
    #X1,X2,y=pickle.load(open('trial_data.pickle','rb'))
    #X=np.concatenate((X1,X2),axis=1)
    #X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.35,random_state=42)
    #print(f'Train_size:{X_train.shape},Test_size:{X_test.shape}')
    df=pd.read_csv('all_images_mat.csv')
    samples=len(df)
    X=np.arange(0,samples)
    y=df['mat']
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.35,random_state=42)
    train_size=len(X_train)
    test_size=len(X_test)
    model=FunRec()
    model.to(device)
    optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
    loss_fn=nn.MSELoss()
    train_loss=[]
    train_accuracy=[]
    train_dataset=Image_dataset(X_train)
    test_dataset=Image_dataset(X_test)
    
    # train_dataset=Images(X_train,y_train)
    # test_dataset=Images(X_test,y_test)
    train_loader=DataLoader(train_dataset,batch_size=100,shuffle=True,num_workers=2)
    test_loader=DataLoader(test_dataset,batch_size=100,shuffle=True,num_workers=2)
    train_loss=[]
    test_loss=[]
    prev_time=time.time()
    for epoch in range(30):
      train_test_loss=[]
      for i,(x1,y) in enumerate(train_loader):
            if time.time()-prev_time>30:
              print(i,train_size/100)
              prev_time=time.time()
            x1=x1.to(device)
            #x2=x2.to(device)
            y=y.to(device)
            optimizer.zero_grad()
            y_pred=model(x1)
            loss=loss_fn(y_pred,y)
            loss.backward()
            optimizer.step()    
      with torch.no_grad():
          true_labels=[]
          final_output=[]
          n_correct=0
          n_samples=0
          train_losses=[]
          for i,(x1,y) in enumerate(train_loader):
            x1=x1.to(device)
           # x2=x2.to(device)
            y=y.to(device)
            y_pred=model(x1)
            loss=loss_fn(y_pred,y)
            train_losses.append(loss.item())
          train_loss.append(np.nanmean(train_losses))
          with open('train_loss.txt','wb') as f:
            np.savetxt(f,np.array(train_loss))
          print(f'epoch:{epoch},loss:{train_loss[-1]}')
          test_losses=[]
          for i,(x1,y) in enumerate(test_loader):
            x1=x1.to(device)
            #x2=x2.to(device)
            y=y.to(device)
            y_pred=model(x1)
            loss=loss_fn(y_pred,y)
            test_losses.append(loss.item())
          test_loss.append(np.nanmean(test_losses))
          with open('test_loss.txt','wb') as f:
            np.savetxt(f,np.array(test_loss))
          print(f'epoch:{epoch},test_loss:{test_loss[-1]}')

          with open(f'{epoch}_model.h5','wb') as f:
            torch.save(model.state_dict(),f)
            


train()







        

mkdir: path should be string, bytes or os.PathLike, not module
1 7700.33


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

class Images(Dataset):
    def __init__(self,X,y):
        self.n_samples=X1.shape[0]

        self.X=torch.from_numpy(X).reshape(-1,2,224,256).float()
        #self.X2=torch.from_numpy(X2).reshape(-1,1,224,256).float()
        self.X1=nn.functional
        self.y=torch.from_numpy(y).reshape(-1,9).float()
        self.y=nn.functional.normalize(self.y,dim=1)

    def __getitem__(self,index):
        return self.X[index],self.y[index]

    def __len__(self):
        return self.n_samples

In [ ]:
places=os.listdir(path)
for place in places:
    if '.' in place:
        places.remove(place)
X1=np.empty((1,1,224,256))
X2=np.empty((1,1,224,256))
y=np.empty(1,9)
for place in places:
  x1,x2,Y=pickle.load(open(f'{place}_data.pickle','rb'))
  X1=np.concatenate((X1,x1),axis=0)
  X2=np.concatenate((X2,x2),axis=0)
  y=np.concatenate((y,Y.reshape(-1,9)),axis=0)

  

In [ ]:
print(X1)

NameError: ignored

In [73]:
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
import pandas as pd
import cv2 as cv

class Image_dataset(Dataset):
  def __init__(self,training_indexs):
    self.path='all_images'
    self.mats=pd.read_csv('all_images_mat.csv')
    self.n_samples=len(training_indexs)
    self.indexes=np.arange(0,self.n_samples)
    self.index_map={}
    for index in self.indexes:
      self.index_map[index]=training_indexs[index]
    
  def __len__(self):
    return self.n_samples

  def __getitem__(self,idx):
    x1=cv.imread(f'all_images/{self.index_map[idx]}_1.jpg',cv.IMREAD_GRAYSCALE)
    x2=cv.imread(f'all_images/{self.index_map[idx]}_2.jpg',cv.IMREAD_GRAYSCALE)
    x=np.array([x1,x2])
    y=self.mats.iloc[idx]['mat'].split(' ')
    y=np.array(y).astype(float)
    y=y/np.max(y)
    return torch.from_numpy(x).float(),torch.from_numpy(y).float()

In [58]:
print(np.arange(0,10))


[0 1 2 3 4 5 6 7 8 9]


In [54]:
y=[]
for place in places:
  pairs=pd.read_csv(f'train/{place}/pair_covisibility.csv')
  pairs=pairs[pairs['covisibility']>=0.1]
  print(place,len(pairs))
  for index,pair in pairs.iterrows():
    fundamental_mat=pair['fundamental_matrix']
    y.append(fundamental_mat)
    




colosseum_exterior 65986
sacre_coeur 34780
taj_mahal 76229
st_peters_square 175024
grand_place_brussels 12316
lincoln_memorial_statue 21401
british_museum 14793
trevi_fountain 243405
sagrada_familia 3832
st_pauls_cathedral 9626
piazza_san_marco 1475
notre_dame_front_facade 322226
temple_nara_japan 22716
pantheon_exterior 48198
brandenburg_gate 50575
buckingham_palace 82085


In [49]:
g=np.ones(9)
f=f'{g}'

print(k)

[1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [55]:
df=pd.DataFrame(y,columns=['mat'])
df.to_csv('all_images_mat.csv')

In [56]:
g=pd.read_csv('all_images_mat.csv')


In [57]:
print(len(g))

1184667
